# Plot timelapse experiments

In [3]:
import napari
import torch
from skimage.measure import regionprops_table, regionprops
import scipy as sp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from matplotlib_scalebar.scalebar import ScaleBar
import gc
import seaborn as sns
from sklearn.mixture import GaussianMixture
from bioio import BioImage
import bioio_nd2
import bioio_tifffile
from bioio.writers import OmeTiffWriter


nucChannel = 0 # red emerin rings
spotChannel = 1 # green spots

# in lucien's original scripts:
# channel 0 is green spots
# channel 1 is red emerin

timeInterval=5 # the interval in minutes between timepoints

pd.set_option('display.max_columns', None)

In [22]:
# on server
base_path = '/mnt/external.data/MeisterLab/jsemple/lhinder/segmentation_Dario/'
df_paths = pd.DataFrame.from_dict({'data_path': [base_path+'/DPY27/1268/20241010_tl/',
                                                base_path+'/SDC1/1273/20241010_tl/']})

output_path=base_path+'/timelapse/'

if not os.path.exists(output_path+"/plots"):
    os.makedirs(output_path+"/plots")

df=pd.DataFrame()
#dist=pd.DataFrame()
file_list = pd.DataFrame()
for path in df_paths['data_path']:
    df_tmp=pd.read_csv(path+"nuclei_analysis_v001.csv")
    df = pd.concat([df,df_tmp])
    #dist_tmp=pd.read_pickle(path+"dist_analysis_v001.pkl")
    #dist = pd.concat([dist,dist_tmp])
    file_list_tmp = pd.read_csv(path+"fileList.csv")
    file_list = pd.concat([file_list,file_list_tmp])

df.head()
dist.head()

,label,bb_dimZ,bb_dimY,bb_dimX,centroid_z,centroid_y,centroid_x,major_axis_length,solidity,mean,median,std,sum,variance,max,min,volume,id,timepoint,intensity_dist_nuclei,intensity_dist_spots,intensity_dist,zproj_spots,zproj_nuclei
0,1,7,94,86,2.595147,636.169992,693.925964,92.824887,0.872761,107.571959,107.0,6.484235,3391206,42.045298,136,84,31525,DPY27_tl_20241010_1268_E_early_3h_5min_5um,1,"[106.1374269005848, 105.04518072289157, 104.00...","[107.00877192982456, 107.25602409638554, 107.9...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
0,1,12,109,105,12.474020,496.429384,694.701003,101.528729,0.830031,107.529925,107.0,6.489472,6841699,42.113248,133,85,63626,DPY27_tl_20241010_1268_E_early_3h_5min_5um,2,"[106.60734463276836, 106.06997084548105, 105.3...","[107.32768361581921, 106.99708454810495, 107.9...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
0,2,12,79,86,11.454992,719.387144,850.006206,90.298296,0.924498,106.020521,106.0,6.155991,5347251,37.896220,131,83,50436,DPY27_tl_20241010_1268_E_early_3h_5min_5um,2,"[107.51757188498402, 106.41447368421052, 105.7...","[106.10543130990415, 105.83223684210526, 106.3...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
0,3,4,92,78,15.633955,713.228358,532.298057,91.633967,0.873681,107.080584,107.0,6.356482,1878943,40.404859,130,84,17547,DPY27_tl_20241010_1268_E_early_3h_5min_5um,2,"[103.67024128686327, 102.51445086705202, 103.1...","[107.02144772117963, 106.85260115606937, 106.3...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
0,1,13,98,95,5.480531,732.856621,844.874120,103.079933,0.919313,107.605859,107.0,6.522628,8190958,42.544679,133,84,76120,DPY27_tl_20241010_1268_E_early_3h_5min_5um,3,"[109.30933333333333, 108.14402173913044, 106.9...","[107.288, 107.23097826086956, 107.581749049429...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


## Add some variables
Radius might scale more linearly than volume
Coefficient of variation is independent of the mean signal

In [5]:
def coefficientOfVariation(std,mean):
    return std/mean


df['cv'] = coefficientOfVariation(df['std'],df['mean'])
df['time'] = df['timepoint']*timeInterval

df.head()


,label,bb_dimZ,bb_dimY,bb_dimX,centroid_z,centroid_y,centroid_x,major_axis_length,solidity,mean,median,std,sum,variance,max,min,volume,id,timepoint,zproj_spots,zproj_nuclei,filename,date,experiment,strain,protein,raw_filepath,denoised_filepath,cv,time
0,1,7,94,86,2.595147,636.169992,693.925964,92.824887,0.872761,107.571959,107.0,6.484235,3391206,42.045298,136,84,31525,DPY27_tl_20241010_1268_E_early_3h_5min_5um,1,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,20241010_1268_E_early_3h_5min_5um,20241010,tl,1268,DPY27,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,0.060278,5
1,1,12,109,105,12.474020,496.429384,694.701003,101.528729,0.830031,107.529925,107.0,6.489472,6841699,42.113248,133,85,63626,DPY27_tl_20241010_1268_E_early_3h_5min_5um,2,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,20241010_1268_E_early_3h_5min_5um,20241010,tl,1268,DPY27,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,0.060350,10
2,2,12,79,86,11.454992,719.387144,850.006206,90.298296,0.924498,106.020521,106.0,6.155991,5347251,37.896220,131,83,50436,DPY27_tl_20241010_1268_E_early_3h_5min_5um,2,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,20241010_1268_E_early_3h_5min_5um,20241010,tl,1268,DPY27,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,0.058064,10
3,3,4,92,78,15.633955,713.228358,532.298057,91.633967,0.873681,107.080584,107.0,6.356482,1878943,40.404859,130,84,17547,DPY27_tl_20241010_1268_E_early_3h_5min_5um,2,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,20241010_1268_E_early_3h_5min_5um,20241010,tl,1268,DPY27,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,0.059362,10
4,1,13,98,95,5.480531,732.856621,844.874120,103.079933,0.919313,107.605859,107.0,6.522628,8190958,42.544679,133,84,76120,DPY27_tl_20241010_1268_E_early_3h_5min_5um,3,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,[[0 0 0 ... 0 0 0]\n [0 0 0 ... 0 0 0]\n [0 0 ...,20241010_1268_E_early_3h_5min_5um,20241010,tl,1268,DPY27,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,/mnt/external.data/MeisterLab/Dario/Imaging/DP...,0.060616,15


# Plot by experiment id

### Plots of coefficient of variation

In [8]:
ids = df.id.unique()

for i in ids:
    df_subset = df[df['id'] == i]
    plt.figure(figsize = (12,5),dpi = 200)
    pointOrder=[t*timeInterval for t in range(df_subset.timepoint.max()+1)] # necessary to deal with early timepoints where no nuclei are detected
    sns.violinplot(data=df_subset, y="cv", x="time",color="skyblue", inner=None,order=pointOrder)
    plt.xticks(rotation=45)
    plt.xlabel("Time (min)")
    plt.ylabel("CV of mSG::"+df_subset.protein.iloc[0]+" intensity")

    # calculate median from first 4 timepoints for horizontal line
    t0_mean = df_subset[df_subset['timepoint'] <= 4]['cv'].median()
    plt.axhline(y=t0_mean, color='grey', linestyle='--', linewidth=0.5)
    
    # get medians points
    medians_df = df_subset.groupby(['timepoint','time'])['cv'].median().reset_index(name='Median')
    plt.scatter(x=medians_df.timepoint, y=medians_df.Median, color='black', s=10)

    # get number of nuclei at each timepoint
    num_nuclei = df_subset.groupby(['timepoint','time'])['cv'].count().reset_index(name='num_nuclei')
    ypos=plt.gca().get_ylim()[0]*1.01
    for n in range(len(num_nuclei)):
        plt.text(x=num_nuclei.timepoint.iloc[n], y=ypos, s=num_nuclei.num_nuclei.iloc[n], color='blue', fontsize=8, ha='center')

    plt.title(i+'::mSG signal upon heatshock') 
    plt.savefig(output_path+"/plots/"+i+"_cv.png")
    plt.close()


### Plots of total fluoresence

In [7]:
ids = df.id.unique()

for i in ids:
    df_subset = df[df['id'] == i]
    plt.figure(figsize = (12,5),dpi = 200)
    pointOrder=[t*timeInterval for t in range(df_subset.timepoint.max()+1)] # necessary to deal with early timepoints where no nuclei are detected
    sns.violinplot(data=df_subset, y="sum", x="time",color="skyblue", inner=None,order=pointOrder)
    plt.xticks(rotation=45)
    plt.xlabel("Time (min)")
    plt.ylabel("Total intensity of mSG::"+df_subset.protein.iloc[0])

    # calculate median from first 4 timepoints for horizontal line
    t0_mean = df_subset[df_subset['timepoint'] <= 4]['sum'].median()
    plt.axhline(y=t0_mean, color='grey', linestyle='--', linewidth=0.5)
    
    # get medians points
    medians_df = df_subset.groupby(['timepoint','time'])['sum'].median().reset_index(name='Median')
    plt.scatter(x=medians_df.timepoint, y=medians_df.Median, color='black', s=10)

    # get number of nuclei at each timepoint
    num_nuclei = df_subset.groupby(['timepoint','time'])['sum'].count().reset_index(name='num_nuclei')
    ypos=plt.gca().get_ylim()[0]*1.1
    for n in range(len(num_nuclei)):
        plt.text(x=num_nuclei.timepoint.iloc[n], y=ypos, s=num_nuclei.num_nuclei.iloc[n], color='blue', fontsize=8, ha='center')

    plt.title(i+'::mSG signal timelapse') 
    plt.savefig(output_path+"/plots/"+i+"_sum.png")
    plt.close()


In [21]:
file_list = pd.read_csv(path+"fileList.csv")
img_path=file_list.denoised_filepath[0]
img_5d = BioImage(img_path, reader=bioio_tifffile.Reader)
img_5d.shape
img = img_5d.get_image_data("XYZC",T=0)


In [114]:

def colorize(im, color, clip_percentile=0.1):
    """
    Helper function to create an RGB image from a single-channel image using a 
    specific color.
    """
    # Check that we do just have a 2D image
    if im.ndim > 2 and im.shape[2] != 1:
        raise ValueError('This function expects a single-channel image!')

    # Rescale the image according to how we want to display it
    im_scaled = im.astype(np.float32) - np.percentile(im, clip_percentile)
    im_scaled = im_scaled / np.percentile(im_scaled, 100 - clip_percentile)
    im_scaled = np.clip(im_scaled, 0, 1)

    # Need to make sure we have a channels dimension for the multiplication to work
    im_scaled = np.atleast_3d(im_scaled)

    # Reshape the color (here, we assume channels last)
    color = np.asarray(color).reshape((1, 1, -1))
    return im_scaled * color

file_list = pd.read_csv(path+"fileList.csv")
index=0
id='SDC1_tl_20241010_1273_E_early_3h_5min_5um'

def get_first_timepoint_images(file_list,index,id):
    img_path=file_list[file_list["id"]==id].raw_filepath.iloc[index]
    if os.path.exists(img_path.replace('.nd2', '_max.tif')):
        img_path=img_path.replace('.nd2', '_max.tif')
        img_5d = BioImage(img_path, reader=bioio_tifffile.Reader)
        toProject=False
    else:
        img_path=file_list.denoised_filepath.iloc[index]
        img_5d = BioImage(img_path, reader=bioio_tifffile.Reader)
        toProject=True

        tdims=img_5d.dims.T
        for t in range(0,4):
            if(toProject):
                img = img_5d.get_image_data("ZXYC",T=t)
                img = np.max(img,axis=0)
            else:
                img = img_5d.get_image_data("XYC",T=t,Z=0)

            im_red = colorize(img[:,:, 0], (1, 0, 1))
            im_green = colorize(img[:,:, 1], (0, 1, 0))

            im_composite = np.clip(im_red + im_green, 0, 1)
            fig, axs = plt.subplots(1, 4, figsize=(12, 3))
            axs[t].axis(False)
            axs[t].imshow(im_composite)
            axs[t].text(0,0,'tp='+str(t),fontsize=12)
            #plt.savefig(output_path+"/plots/"+i+"_t"+str(t)+".png")
            #plt.close()
    plt.show()



In [115]:
index=0
id='SDC1_tl_20241010_1273_E_early_3h_5min_5um'
img_path=file_list[file_list["id"]==id].raw_filepath.iloc[index]
#file_list[file_list["id"]==id]
get_first_timepoint_images(file_list,index,id)
